In [321]:
import requests
import cv2
import json
import numpy as np


In [323]:
# Импортируем api ключи
from api_keys import api_Geosajest, api_Geocoder, api_static

In [356]:
text = 'Где%20поесть' 
types = 'biz'
ll = '37.520905,55.756356'#'37.491709,55.869986'
spn = '0.02,0.02'
distance = 1500


#метро - metro
#парк - biz
#рестораны/каффе - Где%20поесть/Кафе - biz
#Гостиницы - biz

Запрос к карте для получения её изображения

In [115]:
# Создаём функцию запроса для получения карты Яндекс
def requests_map(ll,spn,api):
    response = requests.get(f'https://static-maps.yandex.ru/v1?ll={ll}&spn={spn}&apikey={api}') #ll = 28.978178,41.011218 / spn = 0.01,0.01 / api = api_static
    if response.status_code == 200:
        print("success")
        return response
    else:
        print("error:", str(response.status_code))
    

In [116]:
# Декодируем полученный двоичный код
def decoder(image_data):
    np_arr = np.frombuffer(image_data, np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return img



In [357]:
# Делаем запрос и декодируем
img = decoder(requests_map(ll=ll, spn=spn, api=api_static).content)


success


In [359]:
# Выводим изображение
cv2.imshow('Image_map', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Запрашиваем данные об объектах на карте

In [336]:
# Функция для создания запроса данных с карты
def requests_data(text,types,ll,spn,api):
    response = requests.get(f'https://suggest-maps.yandex.ru/v1/suggest?text={text}%20&types={types}&ll={ll}&spn={spn}&apikey={api}&attrs=uri&print_address=1&ull={ll}&results=10')
    if response.status_code == 200:
        print("success")
        return response
    else:
        print("error:", str(response.status_code))

In [278]:
# Функция для конвертации едениц измерения
def distance_convert(distance_str):
    if distance_str.find('км') != -1:
        distance = float(distance_str[:3]) * 1000
    else:
        distance = float(distance_str[:-2])
    return distance

In [352]:
data_obj_maps = json.loads(requests_data(text=text, types=types, ll=ll, spn=spn, api=api_Geosajest).content)

success


In [353]:
# Перебираем найденные объекты и выводим их если они расположены не дальше задонного значения

num = 0
if str(data_obj_maps).find('results') != -1:
    for i in range (len(data_obj_maps['results'])):
        if distance_convert(data_obj_maps['results'][i]['distance']['text']) <= distance:
            num += 1
            print(data_obj_maps['results'][i]['title']['text'])
    else:
        print(num)
else:
    print(0)


Halal food
Кафе-пекарня
Тануки
Шаурма на районе № 1
Четыре комнаты
Тбилисоба
6


In [354]:
print(len(data_obj_maps['results']))

10


In [355]:
for i in range (len(data_obj_maps['results'])):           
    print(data_obj_maps['results'][i]['title']['text'], distance_convert(data_obj_maps['results'][i]['distance']['text']))

Halal food 314.62
Кафе-пекарня 314.98
Тануки 485.92
Шаурма на районе № 1 360.0
Четыре комнаты 786.92
Тбилисоба 311.37
Где Поесть 20000.0
Где поесть Трактир у Романыча 333000.0
Есть где поесть 159000.0
Лофт кафе 24 часа 16000.0
